In [ ]:
%load_ext autoreload
%autoreload 2
from aoc.lib import load, timing

YEAR = 2024
DAY = 12
TEST = False
TESTDATA = 'RRRRIICCFF\nRRRRIICCCF\nVVRRRCCFFF\nVVRCCCJFFF\nVVVVCJJCFE\nVVIVCCJJEE\nVVIIICJJEE\nMIIIIIJJEE\nMIIISIJEEE\nMMMISSJEEE\n'

In [ ]:
from aoc.lib import CharGrid

class Region:
    def __init__(self, plant):
        self.plant = plant
        self.plots = set()
    
    def merge(self, region):
        assert self.plant == region.plant
        self.plots = self.plots.union(region.plots)
        region.plots = []


class RegionList:
    def __init__(self, grid):
        self.regions = []
        self.index = {}
        self.grid = grid

    def attach(self, plot):
        plant = self.grid[plot]
        up = self.grid.up(plot)
        if plant == self.grid[up]:
            # Attach to existing region
            reg = self.index[up]
        else:
            # Create new region
            self.regions.append(reg := Region(plant))
        return reg

    def update(self, reg, plot):
        self.index[plot] = reg
        reg.plots.add(plot)

    def merge_above(self, plot):
        up = self.grid.up(plot)
        if self.grid.is_inside(up):
            new = self.index[plot]
            old = self.index[up]
            if old != new and old.plant == new.plant:
                for p in self.index:
                    if self.index[p] == old:
                        self.index[p] = new
                    new.merge(old)


@timing
def prepare_data():
    data = load(YEAR, DAY, split_lines=True, test=TESTDATA if TEST else None)
    grid = CharGrid(data['split'])
    regions = RegionList(grid)

    for plot in grid.walk(): # walk() iterates in reading order
        plant = grid[plot]
        
        # New row or mismatch to position to the left: re-attach to area
        if plant != grid[grid.left(plot)]:
            active = regions.attach(plot)

        regions.update(active, plot)
        regions.merge_above(plot)

    return regions

In [ ]:
# Level 1:
def find_edges(grid, region):
    edges = []
    for plot in region.plots:
        for nb in grid.neighbours(plot):
            if grid[nb] != region.plant:
                edges.append(((plot[0] * 3 + nb[0]) / 4, (plot[1] * 3 + nb[1]) / 4))
    return edges

    
@timing
def level1(regions):
    price = 0
    for r in regions.regions:
        area = len(r.plots)
        perimeter = len(find_edges(regions.grid, r))
        price += area * perimeter

    return price


regions = prepare_data()
print(level1(regions))

In [ ]:
# Level 2:
from itertools import pairwise

def count_v_edges(edges):
    count = 0
    for x in set(e[0] for e in edges if e[0] != int(e[0])):
        count += 1
        for a, b in pairwise(sorted(int(e[1]) for e in edges if e[0] == x)):
            if b - a != 1:
                count += 1
    return count


@timing
def level2(regions):
    price = 0
    for r in regions.regions:
        edges = find_edges(regions.grid, r)
        area = len(r.plots)
        perimeter = count_v_edges(edges) + count_v_edges([e[::-1] for e in edges])
        price += area * perimeter
    return price


regions = prepare_data()
print(level2(regions))